In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
import pandas as pd
import os
#import pickle
from utility import *
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import GridSearchCV

In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

def mlp_model_fn(features, labels, mode, params):
  """Model function for MLP."""
  
  #TODO: need works on this
  config = params


  # Input Layer
  
  input_layer = tf.reshape( features["x"], [-1, features["x"].shape[1] ] )
  #print ('feature x', features["x"])
  #print ('feature x shape', features["x"].shape)
  #print ('reshape:', input_layer)
  #print ('reshape shape:', input_layer.shape)
  #trans = tf.string_to_number(input_layer)
  #print ('trans reshape:', trans)
  #print ('reshape shape:', input_layer.shape)


  # Dense Layers
  hidden1 = tf.layers.dense(inputs=input_layer, units=config['n_hidden1'], activation=tf.nn.relu)
  normal_1 = tf.layers.batch_normalization(hidden1, training=mode == tf.estimator.ModeKeys.TRAIN)
 
  drop_h1 = tf.layers.dropout(
      inputs=normal_1, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  hidden2 = tf.layers.dense(inputs=drop_h1, units=config['n_hidden2'], activation=tf.nn.relu)
  drop_h2 = tf.layers.dropout(
      inputs=hidden2, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)
  if config['n_hidden3'] != None:
    hidden3 = tf.layers.dense(inputs=drop_h2, units=config['n_hidden3'], activation=tf.nn.relu)
    drop_h3 = tf.layers.dropout(
        inputs=hidden3, rate=config['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

    logits = tf.layers.dense(inputs=drop_h3, units=config['nclasses'])
  else:
    logits = tf.layers.dense(inputs=drop_h2, units=config['nclasses'])
    
  predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }


  

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=config['nclasses'])
  print (onehot_labels)
  print (logits)
  loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)


  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)

    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [3]:
unused_argv = ['pilot_mlp_newAPI.py', 'data_1p_companded.csv', 'model/', 1024, 512, 256]

filename = unused_argv[1]


if os.path.isfile('ft_to_idx.npy') and os.path.isfile('idx_to_ft.npy') and os.path.isfile('nclasses.npy') and os.path.isfile('group_data.npy'):
    ft_to_idx = np.load('ft_to_idx.npy')
    ft_to_idx = ft_to_idx.item()
    idx_to_ft = np.load('idx_to_ft.npy')
    idx_to_ft = idx_to_ft.item()
    nclasses = np.load('nclasses.npy')
    #f = open('group_data.pkl','r')
    #group_data = pickle.load('group_data.pkl')
    group_data = np.load('group_data.npy') 
    #group_data = np.load('toy_data.npy') # for proof of algo purpose, real use case should use the above line
    group_data = group_data.item()

else:
    ft_to_idx, idx_to_ft, nclasses, group_data = prepare_file(filename)
    np.save("ft_to_idx", ft_to_idx)
    np.save("idx_to_ft", idx_to_ft)
    np.save("nclasses", nclasses)
    #f = open('group_data.pkl','w')
    #pickle.dump(group_data, f)
    np.save("group_data", group_data)

#gen train set
cnt = map(lambda x: len(x),group_data.values())
percentile_40 = np.percentile(cnt, 40)
train, dev ,test = train_dev_test_split(group_data, proportion = [0.6,0.2], thre = percentile_40) # should be 1000 or so
train_data, train_labels = gen_feed(train, ft_to_idx, upper_limit=12000)
del train, group_data
dev_data, dev_labels = gen_feed(dev, ft_to_idx, upper_limit=3000)
del dev
test_data, test_labels = gen_feed(test, ft_to_idx, upper_limit=3000)
del test
train_data = train_data.astype(np.float32) 
dev_data = dev_data.astype(np.float32)
test_data = test_data.astype(np.float32) 

scaler = StandardScaler()
train_data_norm = scaler.fit_transform(train_data)
dev_data_norm = scaler.transform(dev_data)
test_data = scaler.transform(test_data)


In [4]:
from itertools import product
hidden_units = [[1024,1024],[1024,512,256],[256,128,100]]#,[2048,2048],[2048,1024,512]]
batch_sizes = [64,128]#,256,512]
dropouts = [0.4,0.2]

#reset config, defined in previous block
config = {}
config['nclasses'] = len(np.unique(train_labels))
results = []
for hidden, batch_size, dropout in product(hidden_units,batch_sizes,dropouts):
    config['n_hidden1'] = hidden[0]
    config['n_hidden2'] = hidden[1]
    try: config['n_hidden3'] = hidden[2]
    except: config['n_hidden3'] = None
    config['dropout'] = dropout
    # Create the Estimator
    mlp_classifier = tf.estimator.Estimator(
    model_fn=mlp_model_fn, model_dir=None, params=config)

    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=5000)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=batch_size,
      num_epochs=3,
      shuffle=True)

    steps = train_data.shape[0]/batch_size
    print (steps)
    mlp_classifier.train(
      input_fn=train_input_fn,
      steps=None, # 60000
      hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": dev_data},
        y=dev_labels,
        num_epochs=1,
        shuffle=False)

    eval_results = mlp_classifier.evaluate(input_fn=eval_input_fn)
    print ('----setting----')
    print ('hidden units:',hidden)
    print ('batch_size:', batch_size)
    print ('dropout:', dropout)
    print ('----performance----')
    print(eval_results)
    results.append((eval_results['accuracy'], hidden, batch_size, dropout))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x104508310>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpQ9faSO', '_save_summary_steps': 100}
1657.90625
Tensor("one_hot:0", shape=(?, 73), dtype=float32)
Tensor("dense_3/BiasAdd:0", shape=(?, 73), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpQ9faSO/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.06005267  0.00463321  0.00448652 ...,  0.01599341  0.01121983
   

In [5]:
results = sorted(results, key=lambda x: x[0], reverse=True)
print (results)

[(0.2509779, [1024, 512, 256], 64, 0.4), (0.21506853, [1024, 1024], 64, 0.4), (0.18673573, [256, 128, 100], 128, 0.4), (0.18571378, [256, 128, 100], 64, 0.2), (0.183247, [1024, 512, 256], 128, 0.4), (0.18190788, [256, 128, 100], 64, 0.4), (0.18169644, [1024, 512, 256], 64, 0.2), (0.17630476, [1024, 512, 256], 128, 0.2), (0.16774148, [1024, 1024], 64, 0.2), (0.1669662, [256, 128, 100], 128, 0.2), (0.13355887, [1024, 1024], 128, 0.2), (0.085703209, [1024, 1024], 128, 0.4)]


In [7]:
best_param = results[0]
hidden = best_param[1]
config['n_hidden1'] = hidden[0]
config['n_hidden2'] = hidden[1]
try: config['n_hidden3'] = hidden[2]
except: config['n_hidden3'] = None
batch_size = best_param[2]
config['dropout'] = best_param[3]

# Create the Estimator
cnn_classifier = tf.estimator.Estimator(
model_fn=mlp_model_fn, model_dir=None, params=config)

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
  tensors=tensors_to_log, every_n_iter=5000) 

#make full training data
train_full_data = np.vstack((train_data,dev_data))
train_full_labels = np.hstack((train_labels,dev_labels))
train_full_data = scaler.fit_transform(train_full_data)
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": train_full_data},
  y=train_full_labels,
  batch_size=batch_size,
  num_epochs=3,
  shuffle=True)

steps = train_full_data.shape[0]/batch_size
cnn_classifier.train(
  input_fn=train_input_fn,
  steps=None, # 60000
  hooks=[logging_hook])

pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    num_epochs=1,
    shuffle=False)
predictions = mlp_classifier.predict(input_fn=pred_input_fn)
predictions = list(p["classes"] for p in predictions)

print ('----Best setting----')
print ('hidden units:',hidden)
print ('batch_size:', batch_size)
print ('dropout:', dropout)
print ('\n')

target_names = np.unique(test_labels)
target_names = np.sort(target_names)
target_names = [idx_to_ft[i] for i in target_names]
print (classification_report(test_labels, predictions, target_names=target_names))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11c4b6dd0>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpsOapNk', '_save_summary_steps': 100}
Tensor("one_hot:0", shape=(?, 73), dtype=float32)
Tensor("dense_4/BiasAdd:0", shape=(?, 73), dtype=float32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/30/zm33lx8x6673tx7yywr96tr80000gn/T/tmpsOapNk/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.00796972  0.01574038  0.03331136 ...,  0.01107007  0.00892256
   0.00723577]

//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
from sklearn.metrics import accuracy_score
print (accuracy_score(test_labels, predictions,normalize=True))

0.147584847205


In [9]:
train_full_data

array([[ 1.23917294, -0.04110898,  0.05639878, ...,  0.20292   ,
         0.10351221,  0.13790567],
       [ 1.23917294, -0.11398435, -0.07324435, ...,  0.0777875 ,
         0.0059933 ,  0.21702826],
       [ 1.23917294,  0.19936234, -0.03545312, ...,  0.12218257,
         0.06658189,  0.11923256],
       ..., 
       [ 1.23917294, -0.5591538 , -0.63867825, ..., -0.65553826,
        -0.56636339, -0.17270252],
       [ 1.23917294, -0.6293357 , -0.6731177 , ..., -0.68682706,
        -0.65816903, -0.17803136],
       [ 1.23917294, -0.6219483 , -0.62526602, ..., -0.65655553,
        -0.65223938, -0.07372357]], dtype=float32)

In [12]:
train_data[0]

array([ 1.        ,  0.20638844,  0.21380629,  0.26666018,  0.24175856,
        0.22165348,  0.24016015,  0.19883481,  0.23154336,  0.20811196,
        0.20175676,  0.21267353,  0.21606284,  0.22276312,  0.19766   ,
        0.18571211,  0.23317109,  0.19529985,  0.21830767,  0.19824784,
        0.24546783,  0.2046577 ,  0.20638844,  0.20638844,  0.21380629,
        0.2046577 ,  0.20233861,  0.18692394,  0.20117407,  0.19648169,
        0.20523544,  0.20523544,  0.21039893,  0.23317109,  0.21774755,
        0.1893359 ,  0.20233861,  0.20291963,  0.19292533,  0.20925701,
        0.20407917,  0.21606284,  0.22881769,  0.19232944,  0.20523544,
        0.20868486,  0.25071871,  0.28508514,  0.22827059,  0.21493605,
        0.20291963,  0.20233861,  0.20811196,  0.1941144 ,  0.2210976 ,
        0.21437153,  0.21886708,  0.1917326 ,  0.20233861,  0.20982835,
        0.2046577 ,  0.20868486,  0.20349981,  0.19232944,  0.20059054,
        0.20175676,  0.20868486,  0.21942577,  0.22607566,  0.20

In [13]:
train_data_norm[0]

array([  1.23533523e+00,  -4.56794612e-02,   5.19006327e-02,
         2.96029180e-01,   1.85460150e-01,   1.69753969e-01,
         2.45392516e-01,   7.07805604e-02,   1.66426361e-01,
        -1.71903908e-01,  -5.92324197e-01,   1.72599927e-01,
         1.34972319e-01,   3.16191912e-02,   1.00039348e-01,
         5.15589751e-02,   1.98496699e-01,   7.39894435e-02,
         1.88567489e-01,   1.34260848e-01,   2.46700466e-01,
         1.32914647e-01,   1.50730833e-01,   1.71339527e-01,
         1.37569934e-01,   1.40945569e-01,   1.15510762e-01,
         7.41170496e-02,   9.43444595e-02,   1.08665168e-01,
         1.34631366e-01,   1.19016826e-01,  -1.44891453e+00,
         1.80519387e-01,  -4.79914963e-01,  -1.26216710e-02,
         5.16546331e-02,   9.59252864e-02,  -1.36325359e-02,
         5.40427603e-02,  -4.62911688e-02,   3.88316363e-02,
         1.81548670e-01,   3.17496434e-02,  -8.13573077e-02,
        -2.95990944e-01,  -2.21178398e-01,  -3.23330492e-01,
        -2.91605413e-01,